In [2]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Google Drive에 저장한 json 파일 불러오기
import json
from datasets import Dataset
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/sample_data/travel_chatbot_data_241219.json'

# 데이터셋 로드
with open(file_path, 'r') as file:
    data = json.load(file)

# Dataset 생성
train_data = [{"text": f"{row['prompt']}\n{row['response']}"} for row in data]
dataset = Dataset.from_dict({"text": [item["text"] for item in train_data]})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 1. pad_token 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # eos_token을 pad_token으로 설정
    model.resize_token_embeddings(len(tokenizer))  # 모델의 임베딩 크기 재조정

# 데이터 전처리 함수
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",  # Padding을 사용
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # labels 추가
    return tokenized

# 데이터 전처리
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Fine-tuning 설정
training_args = TrainingArguments(
    output_dir="./kogpt2-results",
    evaluation_strategy="no",  # 평가 비활성화
    report_to="none",  # wandb 비활성화
    learning_rate=5e-5,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    save_steps=200,
    save_total_limit=2,
    logging_dir="./logs"
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# 모델 학습
trainer.train()

# 모델 저장
model.save_pretrained("./fine_tuned_kogpt2")
tokenizer.save_pretrained("./fine_tuned_kogpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/723 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-1d62dd4f8f17>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,1.178700
1000,0.027600
1500,0.015300
2000,0.012500


('./fine_tuned_kogpt2/tokenizer_config.json',
 './fine_tuned_kogpt2/special_tokens_map.json',
 './fine_tuned_kogpt2/vocab.json',
 './fine_tuned_kogpt2/merges.txt',
 './fine_tuned_kogpt2/added_tokens.json',
 './fine_tuned_kogpt2/tokenizer.json')

In [ ]:
from transformers import pipeline

# Fine-tuned 모델 로드
model_name = "./fine_tuned_kogpt2"
qa_pipeline = pipeline("text-generation", model=model_name, tokenizer=model_name)

# 질문 생성
question = "서울 여행지 추천해줘"
# 텍스트 생성
result = qa_pipeline(
    question,
    max_length=100,  # 답변의 최대 길이 제한
    num_return_sequences=1,  # 하나의 답변만 생성
    top_p=1,  # 확률 누적 분포 제한
    temperature=1.2,  # 무작위성 조정
    repetition_penalty=1.8  # 반복 패널티 추가
)

# 생성된 텍스트 후처리
generated_text = result[0]["generated_text"]
print('\n',generated_text)

# 온점 기준으로 텍스트 잘라서 마지막 문장 제거
if "." in generated_text:
    processed_text = generated_text[:generated_text.rfind(".") + 1]
else:
    processed_text = generated_text  # 온점이 없으면 원본 텍스트 유지
print('\n',processed_text)


# 첫 번째 온점 뒤에서 두 번째 온점 앞까지 가져오기
if "." in generated_text:
    first_dot = generated_text.find(".")  # 첫 번째 온점 위치
    second_dot = generated_text.find(".", first_dot + 1)  # 두 번째 온점 위치
    if second_dot != -1:  # 두 번째 온점이 존재할 경우
        processed_text2 = generated_text[first_dot + 1:second_dot].strip()  # 첫 번째 온점 뒤 ~ 두 번째 온점 앞
    else:
        processed_text2 = ""  # 두 번째 온점이 없으면 빈 문자열
else:
    processed_text2 = ""  # 온점이 없으면 빈 문자열
print('\n',processed_text2)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



 서울 여행지 추천해줘.
서울의 추천 관광지는 전쟁기념관입니다. 6·25전쟁납북자기념관은 경기 양주시 한탄강로 396에 있으며 납 북자 가족들이 1946년 9월 22일부터 2000년 6월 13일까지 남한에서 거주하던 현실을 인식하게 할 수 있는 전시실이 있다. 고려청자의 집, 용문산의 송림이 울창하여 경치가 아름답고 아버지는 연거포 초등학교 3년에 용산에서 유학하세요 용산구 한강변 새남터에서 처형되었기 때문에 어머니를 그리워하며 뛰어놀았던 추억이

 서울 여행지 추천해줘.
서울의 추천 관광지는 전쟁기념관입니다. 6·25전쟁납북자기념관은 경기 양주시 한탄강로 396에 있으며 납 북자 가족들이 1946년 9월 22일부터 2000년 6월 13일까지 남한에서 거주하던 현실을 인식하게 할 수 있는 전시실이 있다.

 서울의 추천 관광지는 전쟁기념관입니다
